In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [2]:
def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidationPoKolei2(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

def crossValidationPoKolei(clf, data, dataClass):
    err = 0
    k = len(data)
    for i in range(k):
        X_train = data[i][0]
        y_train = dataClass[i][0]
        X_test = data[i][1]
        y_test = dataClass[i][1]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def generujPodzialy(tab, p = 0.75):
    podzial = []
    n = len(tab)
    k = int(10*n/100)
    dlugosc = int(4/10*n)
    dlugoscUczaca = int(4/10*p*n)
    for i in range(6):
        uczace = tab[np.arange(i*k, min(i*k + dlugoscUczaca, n))]
        testowe = tab[np.arange(min(i*k + dlugoscUczaca, n), min((i*k + dlugosc), n))]
        podzial.append([uczace, testowe])
    return podzial

In [30]:
gammas = [1e-6, 1e-4, 1e-3, 1e-2, 1, 1e1, 1e2]
Cs = [1e-6, 1e-4, 1e-2, 1e-1, 1, 1e1, 1e2, 1e4]
tabParametrow = np.zeros((len(gammas)*len(Cs),2), dtype=np.float64)
i = 0
for gamma in gammas:
    for C in Cs:
        tabParametrow[i][0] = gamma
        tabParametrow[i][1] = C
        i += 1

In [4]:
zmiana = np.load("zmiana.npy")
podzialZmiana = generujPodzialy(zmiana)

In [31]:
def test(tablica, tablica2, tablica3, tablica4, tablica5, podzialZmiana, svm):
    wynik1 = crossValidationPoKolei(svm, tablica, podzialZmiana)
    wynik2 = crossValidationPoKolei(svm, tablica2, podzialZmiana)
    wynik3 = crossValidationPoKolei(svm, tablica3, podzialZmiana)
    wynik4= crossValidationPoKolei(svm, tablica4, podzialZmiana) 
    wynik5 = crossValidationPoKolei(svm, tablica5, podzialZmiana) 
    return wynik1, wynik2, wynik3, wynik4, wynik5

def wylicz(TAB, podzialZmiana, tabParametrow):
    tab, tabS, tabN, tabSN, tabNS = standaryzowanie(TAB)
    podzialTab = generujPodzialy(tab)
    podzialTabS = generujPodzialy(tabS)
    podzialTabN = generujPodzialy(tabN)
    podzialTabSN = generujPodzialy(tabSN)
    podzialTabnS = generujPodzialy(tabNS)
    
    tabWynikow = np.zeros((len(tabParametrow),5), dtype=np.float32)
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C, random_state=1)
        tabWynikow[i][0], tabWynikow[i][1], tabWynikow[i][2], tabWynikow[i][3], tabWynikow[i][4] = test(
            podzialTab, podzialTabS, podzialTabN, podzialTabSN, podzialTabnS, podzialZmiana, svm)
        i  += 1
    return tabWynikow

def wypisz(tabWynikow):
    print('Czyste     S      N     S->N    N->S')
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        wynik1, wynik2, wynik3, wynik4, wynik5 = tabWynikow[i]
        odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
        if (wynik1 > 0.52 or wynik2 > 0.52 or wynik3 > 0.52 or wynik4 > 0.52 or wynik5 > 0.52):
            odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
        print(odpowiedz, "- dla: g:", gamma, "C:", C) 
        i += 1

In [32]:
ticki_M_fPierw = np.load("ticki_M_fPierw.npy")
wynikiTicki_M_fPierw = wylicz(ticki_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiTicki_M_fPierwTest2", wynikiTicki_M_fPierw)
wypisz(wynikiTicki_M_fPierw)

55221.03191423416
Czyste     S      N     S->N    N->S
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 1e-06
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 0.0001
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 0.01
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 0.1
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 1.0
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 10.0
0.5011  0.4942  0.4829  0.4953  0.4913 - dla: g: 1e-06 C: 100.0
0.5011  0.4920  0.4829  0.4953  0.4956 - dla: g: 1e-06 C: 10000.0
0.5011  0.4942  0.4829  0.4953  0.4935 - dla: g: 0.0001 C: 1e-06
0.5011  0.4942  0.4829  0.4953  0.4935 - dla: g: 0.0001 C: 0.0001
0.5011  0.4942  0.4829  0.4953  0.4935 - dla: g: 0.0001 C: 0.01
0.5011  0.4942  0.4829  0.4953  0.4935 - dla: g: 0.0001 C: 0.1
0.5011  0.4942  0.4829  0.4953  0.4935 - dla: g: 0.0001 C: 1.0
0.5011  0.5015  0.4829  0.4953  0.4996 - dla: g: 0.0001 C: 10.0
0.5011  0.4924  0.4829  0.4953  0.4942 - dla: g: 0.

In [16]:
t = time.time()
gmm_5_M_fPierw = np.load("gmm_5_M_fPierw.npy")
wynikiGmm_5_M_fPierw = wylicz(gmm_5_M_fPierw, podzialZmiana, tabParametrow)
print (time.time() -t)
np.save("wynikiGmm_5_M_fPierwTest2", wynikiGmm_5_M_fPierw)
wypisz(wynikiGmm_5_M_fPierw)

1069.2221548557281
Czyste     S      N     S->N    N->S
0.5011  0.5018  0.4978  0.4931  0.4971 - dla: g: 1e-06 C: 1e-06
0.5015  0.5018  0.4978  0.4931  0.4971 - dla: g: 1e-06 C: 0.0001
0.5015  0.5018  0.4978  0.4931  0.4971 - dla: g: 1e-06 C: 0.01
0.5015  0.5018  0.4978  0.4931  0.4971 - dla: g: 1e-06 C: 0.1
0.5015  0.5018  0.4978  0.4931  0.4971 - dla: g: 1e-06 C: 1.0
0.5015  0.5018  0.4978  0.4931  0.4971 - dla: g: 1e-06 C: 10.0
0.5015  0.5018  0.4978  0.4931  0.5025 - dla: g: 1e-06 C: 100.0
0.5015  0.5109  0.4978  0.4931  0.5189 - dla: g: 1e-06 C: 10000.0
0.5015  0.5033  0.4978  0.4935  0.5033 - dla: g: 0.0001 C: 1e-06
0.5015  0.5033  0.4978  0.4935  0.5033 - dla: g: 0.0001 C: 0.0001
0.5015  0.5033  0.4978  0.4935  0.5033 - dla: g: 0.0001 C: 0.01
0.5015  0.5033  0.4978  0.4935  0.5033 - dla: g: 0.0001 C: 0.1
0.5015  0.5029  0.4978  0.4935  0.5029 - dla: g: 0.0001 C: 1.0
0.5015  0.4949  0.4978  0.4935  0.4956 - dla: g: 0.0001 C: 10.0
0.5015  0.5182  0.4978  0.4935  0.5171 - dla: g: 0

In [17]:
gmm_10_M_fPierw = np.load("gmm_10_M_fPierw.npy")
wynikiGmm_10_M_fPierw = wylicz(gmm_10_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_10_M_fPierwTest2", wynikiGmm_10_M_fPierw)
wypisz(wynikiGmm_10_M_fPierw)

Czyste     S      N     S->N    N->S
0.4985  0.4938  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 1e-06
0.4996  0.4938  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 0.0001
0.4996  0.4938  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 0.01
0.4996  0.4938  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 0.1
0.4996  0.4938  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 1.0
0.4996  0.4938  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 10.0
0.4996  0.4993  0.4996  0.4891  0.4855 - dla: g: 1e-06 C: 100.0
0.4996  0.5087  0.4996  0.4891  0.5062 - dla: g: 1e-06 C: 10000.0
0.4996  0.4869  0.4996  0.4891  0.4844 - dla: g: 0.0001 C: 1e-06
0.4996  0.4869  0.4996  0.4891  0.4844 - dla: g: 0.0001 C: 0.0001
0.4996  0.4869  0.4996  0.4891  0.4844 - dla: g: 0.0001 C: 0.01
0.4996  0.4869  0.4996  0.4891  0.4844 - dla: g: 0.0001 C: 0.1
0.4996  0.4949  0.4996  0.4891  0.4847 - dla: g: 0.0001 C: 1.0
0.4996  0.5022  0.4996  0.4891  0.5025 - dla: g: 0.0001 C: 10.0
0.4996  0.5065  0.4996  0.4891  0.5138 - dla: g: 0.0001 C: 100.0
0.49

In [18]:
gmm_15_M_fPierw = np.load("gmm_15_M_fPierw.npy")
wynikiGmm_15_M_fPierw = wylicz(gmm_15_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_15_M_fPierwTest2", wynikiGmm_15_M_fPierw)
wypisz(wynikiGmm_15_M_fPierw)

Czyste     S      N     S->N    N->S
0.4840  0.4956  0.4844  0.4964  0.4982 - dla: g: 1e-06 C: 1e-06
0.4840  0.4956  0.4844  0.4964  0.4982 - dla: g: 1e-06 C: 0.0001
0.4840  0.4956  0.4844  0.4964  0.4982 - dla: g: 1e-06 C: 0.01
0.4840  0.4956  0.4844  0.4964  0.4982 - dla: g: 1e-06 C: 0.1
0.4840  0.4956  0.4844  0.4964  0.4982 - dla: g: 1e-06 C: 1.0
0.4840  0.4956  0.4844  0.4964  0.4982 - dla: g: 1e-06 C: 10.0
0.4840  0.4946  0.4844  0.4964  0.4993 - dla: g: 1e-06 C: 100.0
0.4840  0.4902  0.4844  0.4964  0.4906 - dla: g: 1e-06 C: 10000.0
0.4840  0.4956  0.4844  0.4964  0.5007 - dla: g: 0.0001 C: 1e-06
0.4840  0.4956  0.4844  0.4964  0.5007 - dla: g: 0.0001 C: 0.0001
0.4840  0.4956  0.4844  0.4964  0.5007 - dla: g: 0.0001 C: 0.01
0.4840  0.4956  0.4844  0.4964  0.5007 - dla: g: 0.0001 C: 0.1
0.4840  0.4956  0.4844  0.4964  0.4993 - dla: g: 0.0001 C: 1.0
0.4840  0.4877  0.4844  0.4964  0.4815 - dla: g: 0.0001 C: 10.0
0.4840  0.4927  0.4844  0.4964  0.4978 - dla: g: 0.0001 C: 100.0
0.48

In [19]:
gmm_20_M_fPierw = np.load("gmm_20_M_fPierw.npy")
wynikiGmm_20_M_fPierw = wylicz(gmm_20_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_20_M_fPierwTest2", wynikiGmm_20_M_fPierw)
wypisz(wynikiGmm_20_M_fPierw)

Czyste     S      N     S->N    N->S
0.4895  0.4844  0.4833  0.5120  0.4985 - dla: g: 1e-06 C: 1e-06
0.4895  0.4844  0.4833  0.5120  0.4985 - dla: g: 1e-06 C: 0.0001
0.4895  0.4844  0.4833  0.5120  0.4985 - dla: g: 1e-06 C: 0.01
0.4895  0.4844  0.4833  0.5120  0.4985 - dla: g: 1e-06 C: 0.1
0.4895  0.4844  0.4833  0.5120  0.4985 - dla: g: 1e-06 C: 1.0
0.4895  0.4844  0.4833  0.5120  0.4985 - dla: g: 1e-06 C: 10.0
0.4895  0.4887  0.4833  0.5120  0.4993 - dla: g: 1e-06 C: 100.0
0.4895  0.5007  0.4833  0.5120  0.4967 - dla: g: 1e-06 C: 10000.0
0.4895  0.4913  0.4833  0.5120  0.4971 - dla: g: 0.0001 C: 1e-06
0.4895  0.4913  0.4833  0.5120  0.4971 - dla: g: 0.0001 C: 0.0001
0.4895  0.4913  0.4833  0.5120  0.4971 - dla: g: 0.0001 C: 0.01
0.4895  0.4913  0.4833  0.5120  0.4971 - dla: g: 0.0001 C: 0.1
0.4895  0.4906  0.4833  0.5120  0.5036 - dla: g: 0.0001 C: 1.0
0.4895  0.5000  0.4833  0.5120  0.4913 - dla: g: 0.0001 C: 10.0
0.4895  0.5051  0.4833  0.5120  0.5022 - dla: g: 0.0001 C: 100.0
0.48

In [20]:
gmm_25_M_fPierw = np.load("gmm_25_M_fPierw.npy")
wynikiGmm_25_M_fPierw = wylicz(gmm_25_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_25_M_fPierwTest2", wynikiGmm_25_M_fPierw)
wypisz(wynikiGmm_25_M_fPierw)

Czyste     S      N     S->N    N->S
0.4949  0.4887  0.4927  0.4931  0.4913 - dla: g: 1e-06 C: 1e-06
0.4949  0.4887  0.4927  0.4931  0.4913 - dla: g: 1e-06 C: 0.0001
0.4949  0.4887  0.4927  0.4931  0.4913 - dla: g: 1e-06 C: 0.01
0.4949  0.4887  0.4927  0.4931  0.4913 - dla: g: 1e-06 C: 0.1
0.4949  0.4887  0.4927  0.4931  0.4913 - dla: g: 1e-06 C: 1.0
0.4949  0.4887  0.4927  0.4931  0.4913 - dla: g: 1e-06 C: 10.0
0.4949  0.4869  0.4927  0.4931  0.4902 - dla: g: 1e-06 C: 100.0
0.4949  0.4942  0.4927  0.4931  0.4935 - dla: g: 1e-06 C: 10000.0
0.4949  0.4895  0.4927  0.4931  0.4931 - dla: g: 0.0001 C: 1e-06
0.4949  0.4895  0.4927  0.4931  0.4931 - dla: g: 0.0001 C: 0.0001
0.4949  0.4895  0.4927  0.4931  0.4931 - dla: g: 0.0001 C: 0.01
0.4949  0.4895  0.4927  0.4931  0.4931 - dla: g: 0.0001 C: 0.1
0.4949  0.4898  0.4927  0.4931  0.4916 - dla: g: 0.0001 C: 1.0
0.4949  0.4931  0.4927  0.4931  0.4956 - dla: g: 0.0001 C: 10.0
0.4949  0.4895  0.4927  0.4931  0.4895 - dla: g: 0.0001 C: 100.0
0.49

In [21]:
gmm_30_M_fPierw = np.load("gmm_30_M_fPierw.npy")
wynikiGmm_30_M_fPierw = wylicz(gmm_30_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_30_M_fPierwTest2", wynikiGmm_30_M_fPierw)
wypisz(wynikiGmm_30_M_fPierw)

Czyste     S      N     S->N    N->S
0.4862  0.4916  0.4847  0.4800  0.4844 - dla: g: 1e-06 C: 1e-06
0.4862  0.4916  0.4847  0.4800  0.4844 - dla: g: 1e-06 C: 0.0001
0.4862  0.4916  0.4847  0.4800  0.4844 - dla: g: 1e-06 C: 0.01
0.4862  0.4916  0.4847  0.4800  0.4844 - dla: g: 1e-06 C: 0.1
0.4862  0.4916  0.4847  0.4800  0.4844 - dla: g: 1e-06 C: 1.0
0.4862  0.4916  0.4847  0.4800  0.4844 - dla: g: 1e-06 C: 10.0
0.4862  0.4920  0.4847  0.4800  0.4949 - dla: g: 1e-06 C: 100.0
0.4862  0.4924  0.4847  0.4800  0.4891 - dla: g: 1e-06 C: 10000.0
0.4862  0.4927  0.4847  0.4800  0.4808 - dla: g: 0.0001 C: 1e-06
0.4862  0.4927  0.4847  0.4800  0.4808 - dla: g: 0.0001 C: 0.0001
0.4862  0.4927  0.4847  0.4800  0.4808 - dla: g: 0.0001 C: 0.01
0.4862  0.4927  0.4847  0.4800  0.4808 - dla: g: 0.0001 C: 0.1
0.4862  0.4927  0.4847  0.4800  0.4811 - dla: g: 0.0001 C: 1.0
0.4862  0.4847  0.4847  0.4800  0.4880 - dla: g: 0.0001 C: 10.0
0.4862  0.4942  0.4847  0.4800  0.4887 - dla: g: 0.0001 C: 100.0
0.48

In [22]:
gmm_35_M_fPierw = np.load("gmm_35_M_fPierw.npy")
wynikiGmm_35_M_fPierw = wylicz(gmm_35_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_35_M_fPierwTest2", wynikiGmm_35_M_fPierw)
wypisz(wynikiGmm_35_M_fPierw)

Czyste     S      N     S->N    N->S
0.4833  0.4891  0.4815  0.4847  0.4887 - dla: g: 1e-06 C: 1e-06
0.4826  0.4891  0.4811  0.4847  0.4887 - dla: g: 1e-06 C: 0.0001
0.4826  0.4891  0.4811  0.4847  0.4887 - dla: g: 1e-06 C: 0.01
0.4826  0.4891  0.4811  0.4847  0.4887 - dla: g: 1e-06 C: 0.1
0.4826  0.4891  0.4811  0.4847  0.4887 - dla: g: 1e-06 C: 1.0
0.4826  0.4891  0.4811  0.4847  0.4887 - dla: g: 1e-06 C: 10.0
0.4826  0.5040  0.4811  0.4847  0.4873 - dla: g: 1e-06 C: 100.0
0.4826  0.4891  0.4811  0.4847  0.4946 - dla: g: 1e-06 C: 10000.0
0.4826  0.4913  0.4811  0.4847  0.4887 - dla: g: 0.0001 C: 1e-06
0.4826  0.4913  0.4811  0.4847  0.4887 - dla: g: 0.0001 C: 0.0001
0.4826  0.4913  0.4811  0.4847  0.4887 - dla: g: 0.0001 C: 0.01
0.4826  0.4913  0.4811  0.4847  0.4887 - dla: g: 0.0001 C: 0.1
0.4826  0.5033  0.4811  0.4847  0.4880 - dla: g: 0.0001 C: 1.0
0.4826  0.4916  0.4811  0.4847  0.4873 - dla: g: 0.0001 C: 10.0
0.4826  0.4924  0.4811  0.4847  0.4906 - dla: g: 0.0001 C: 100.0
0.48

In [23]:
gmm_40_M_fPierw = np.load("gmm_40_M_fPierw.npy")
wynikiGmm_40_M_fPierw = wylicz(gmm_40_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_40_M_fPierwTest2", wynikiGmm_40_M_fPierw)
wypisz(wynikiGmm_40_M_fPierw)

Czyste     S      N     S->N    N->S
0.4779  0.4829  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 1e-06
0.4771  0.4829  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 0.0001
0.4771  0.4829  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 0.01
0.4771  0.4829  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 0.1
0.4771  0.4829  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 1.0
0.4771  0.4829  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 10.0
0.4771  0.4818  0.4782  0.4837  0.4887 - dla: g: 1e-06 C: 100.0
0.4771  0.4913  0.4782  0.4837  0.4906 - dla: g: 1e-06 C: 10000.0
0.4771  0.4858  0.4782  0.4837  0.4895 - dla: g: 0.0001 C: 1e-06
0.4771  0.4858  0.4782  0.4837  0.4895 - dla: g: 0.0001 C: 0.0001
0.4771  0.4858  0.4782  0.4837  0.4895 - dla: g: 0.0001 C: 0.01
0.4771  0.4858  0.4782  0.4837  0.4895 - dla: g: 0.0001 C: 0.1
0.4771  0.4858  0.4782  0.4837  0.4895 - dla: g: 0.0001 C: 1.0
0.4771  0.4862  0.4782  0.4837  0.4815 - dla: g: 0.0001 C: 10.0
0.4771  0.4938  0.4782  0.4837  0.4913 - dla: g: 0.0001 C: 100.0
0.47

In [24]:
gmm_50_M_fPierw = np.load("gmm_50_M_fPierw.npy")
wynikiGmm_50_M_fPierw = wylicz(gmm_50_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_50_M_fPierwTest2", wynikiGmm_50_M_fPierw)
wypisz(wynikiGmm_50_M_fPierw)

Czyste     S      N     S->N    N->S
0.4837  0.4858  0.4851  0.4851  0.4847 - dla: g: 1e-06 C: 1e-06
0.4837  0.4858  0.4851  0.4851  0.4847 - dla: g: 1e-06 C: 0.0001
0.4837  0.4858  0.4851  0.4851  0.4847 - dla: g: 1e-06 C: 0.01
0.4837  0.4858  0.4851  0.4851  0.4847 - dla: g: 1e-06 C: 0.1
0.4837  0.4858  0.4851  0.4851  0.4847 - dla: g: 1e-06 C: 1.0
0.4837  0.4851  0.4851  0.4851  0.4840 - dla: g: 1e-06 C: 10.0
0.4837  0.4844  0.4851  0.4851  0.4840 - dla: g: 1e-06 C: 100.0
0.4837  0.4982  0.4851  0.4851  0.4956 - dla: g: 1e-06 C: 10000.0
0.4837  0.4851  0.4851  0.4851  0.4840 - dla: g: 0.0001 C: 1e-06
0.4837  0.4851  0.4851  0.4851  0.4840 - dla: g: 0.0001 C: 0.0001
0.4837  0.4851  0.4851  0.4851  0.4840 - dla: g: 0.0001 C: 0.01
0.4837  0.4851  0.4851  0.4851  0.4840 - dla: g: 0.0001 C: 0.1
0.4837  0.4844  0.4851  0.4851  0.4837 - dla: g: 0.0001 C: 1.0
0.4837  0.4967  0.4851  0.4851  0.4989 - dla: g: 0.0001 C: 10.0
0.4837  0.4964  0.4851  0.4851  0.4956 - dla: g: 0.0001 C: 100.0
0.48

In [25]:
gmm_60_M_fPierw = np.load("gmm_60_M_fPierw.npy")
wynikiGmm_60_M_fPierw = wylicz(gmm_60_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_60_M_fPierwTest2", wynikiGmm_60_M_fPierw)
wypisz(wynikiGmm_60_M_fPierw)

Czyste     S      N     S->N    N->S
0.4858  0.4851  0.4880  0.4902  0.4855 - dla: g: 1e-06 C: 1e-06
0.4858  0.4851  0.4880  0.4902  0.4855 - dla: g: 1e-06 C: 0.0001
0.4858  0.4851  0.4880  0.4902  0.4855 - dla: g: 1e-06 C: 0.01
0.4858  0.4851  0.4880  0.4902  0.4855 - dla: g: 1e-06 C: 0.1
0.4858  0.4851  0.4880  0.4902  0.4855 - dla: g: 1e-06 C: 1.0
0.4858  0.4847  0.4880  0.4902  0.4855 - dla: g: 1e-06 C: 10.0
0.4858  0.4851  0.4880  0.4902  0.4851 - dla: g: 1e-06 C: 100.0
0.4858  0.5004  0.4880  0.4902  0.5018 - dla: g: 1e-06 C: 10000.0
0.4858  0.4858  0.4880  0.4902  0.4844 - dla: g: 0.0001 C: 1e-06
0.4858  0.4858  0.4880  0.4902  0.4844 - dla: g: 0.0001 C: 0.0001
0.4858  0.4858  0.4880  0.4902  0.4844 - dla: g: 0.0001 C: 0.01
0.4858  0.4858  0.4880  0.4902  0.4844 - dla: g: 0.0001 C: 0.1
0.4858  0.4858  0.4880  0.4902  0.4844 - dla: g: 0.0001 C: 1.0
0.4858  0.4964  0.4880  0.4902  0.4956 - dla: g: 0.0001 C: 10.0
0.4858  0.5091  0.4880  0.4902  0.5018 - dla: g: 0.0001 C: 100.0
0.48

In [33]:
gmm_70_M_fPierw = np.load("gmm_70_M_fPierw.npy")
wynikiGmm_70_M_fPierw = wylicz(gmm_70_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_70_M_fPierwTest2", wynikiGmm_70_M_fPierw)
wypisz(wynikiGmm_70_M_fPierw)

Czyste     S      N     S->N    N->S
0.4789  0.4822  0.4779  0.4913  0.4847 - dla: g: 1e-06 C: 1e-06
0.4779  0.4822  0.4779  0.4913  0.4847 - dla: g: 1e-06 C: 0.0001
0.4779  0.4822  0.4779  0.4913  0.4847 - dla: g: 1e-06 C: 0.01
0.4779  0.4822  0.4779  0.4913  0.4847 - dla: g: 1e-06 C: 0.1
0.4779  0.4822  0.4779  0.4913  0.4847 - dla: g: 1e-06 C: 1.0
0.4779  0.4822  0.4779  0.4913  0.4847 - dla: g: 1e-06 C: 10.0
0.4779  0.4858  0.4779  0.4913  0.4844 - dla: g: 1e-06 C: 100.0
0.4779  0.5036  0.4779  0.4913  0.4935 - dla: g: 1e-06 C: 10000.0
0.4779  0.4833  0.4779  0.4913  0.4844 - dla: g: 0.0001 C: 1e-06
0.4779  0.4833  0.4779  0.4913  0.4844 - dla: g: 0.0001 C: 0.0001
0.4779  0.4833  0.4779  0.4913  0.4844 - dla: g: 0.0001 C: 0.01
0.4779  0.4833  0.4779  0.4913  0.4844 - dla: g: 0.0001 C: 0.1
0.4779  0.4840  0.4779  0.4913  0.4840 - dla: g: 0.0001 C: 1.0
0.4779  0.4960  0.4779  0.4913  0.4924 - dla: g: 0.0001 C: 10.0
0.4779  0.5033  0.4779  0.4913  0.4935 - dla: g: 0.0001 C: 100.0
0.47

In [34]:
gmm_80_M_fPierw = np.load("gmm_80_M_fPierw.npy")
wynikiGmm_80_M_fPierw = wylicz(gmm_80_M_fPierw, podzialZmiana, tabParametrow)
np.save("wynikiGmm_80_M_fPierwTest2", wynikiGmm_80_M_fPierw)
wypisz(wynikiGmm_80_M_fPierw)

Czyste     S      N     S->N    N->S
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 1e-06
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 0.0001
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 0.01
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 0.1
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 1.0
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 10.0
0.4793  0.4858  0.4782  0.4913  0.4840 - dla: g: 1e-06 C: 100.0
0.4793  0.4924  0.4782  0.4913  0.4877 - dla: g: 1e-06 C: 10000.0
0.4793  0.4866  0.4782  0.4913  0.4847 - dla: g: 0.0001 C: 1e-06
0.4793  0.4866  0.4782  0.4913  0.4847 - dla: g: 0.0001 C: 0.0001
0.4793  0.4866  0.4782  0.4913  0.4847 - dla: g: 0.0001 C: 0.01
0.4793  0.4866  0.4782  0.4913  0.4847 - dla: g: 0.0001 C: 0.1
0.4793  0.4866  0.4782  0.4913  0.4847 - dla: g: 0.0001 C: 1.0
0.4793  0.4938  0.4782  0.4913  0.5007 - dla: g: 0.0001 C: 10.0
0.4793  0.4949  0.4782  0.4913  0.4869 - dla: g: 0.0001 C: 100.0
0.47